<a href="https://colab.research.google.com/github/JuanAcevedo08/ComputerVision/blob/main/guia_fintuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1) Cargar los datos**

Estos pueden ser generados por ti o por el cliente importante que contengan como siempre en temas de computer vision

In [ ]:
%%capture
!unrar x "/content/defectos_de_soldaduras.rar"

# **2) Dependencias**

In [ ]:
%%capture
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# **3) Entrenamiento**

In [ ]:
#Cargar un modelo preentrenado
model = YOLO('yolo11n.pt')

In [ ]:
#Definir el path de yaml que nos va indicar donde están los ddatos de train, test, val, unmero de clases y clases a evaluar(Es un archivo de configuracion)
yaml_path = '/content/defectos_de_soldaduras/data.yaml'

## **1) 🪐Opción 1 confg Modelo por DEFAULT**

In [ ]:
model.train(
    data=yaml_path, #De donde van a salir los datos
    epochs=10,#Cantidad de iteraciones
    imgsz=640, #Tamaño de las imagenes (IMagenes con el que modelo fue entrenado)
    augment=True #aumentar imagenes con datos alterados para permitir mejor aprendizaje
)

## **2) 🪐Opción 2 callback de early stopping**

Usa la pateicne en la parte de loss validation , si no mejora lo frena

In [ ]:
#Ayuda para evitar el overfitting para saber cuantos epochs necesita el set de datos o si el modelo no está aprendiendo
model.train(
    data=yaml_path, #De donde van a salir los datos
    epochs=20,#Cantidad de iteraciones
    imgsz=640, #Tamaño de las imagenes (IMagenes con el que modelo fue entrenado)
    augment=True, #aumentar imagenes con datos alterados para permitir mejor aprendizaje
    patience=5
)

## **3) 🪐Opción 3 Custom Data Augmentation**

In [ ]:
#Podemos usar parametros para poder alterar las imagenes , como saturación, rotacion y demas para mejorar las predicciones
model = YOLO('yolo11n.pt')

model.train(
    data=yaml_path,
    epochs=10,
    imgsz=640,
    patience=5,
    hsv_h=0.0015, #Cambiar el color
    hsv_s=0.7, #Ajusta la saturacion
    hsv_v=0.0015, #Ajusta el brillo
    flipud=0.5, #Voltea verticalment econ el 50%  de probabilidad
    fliplr=0.5, #Voltea horizontal econ el 50%  de probabilidad
    mosaic=1.0, #Mezcla de imagenes
    mixup=0.5, #mezcla dos imagenes con probabilidad 50%
    perspective=0.0005, #Transformacion prespectiva
    degrees=5.0, #Rotacion 5 grados
    shear=2.0, #inclinación de la iamgen
    scale=0.5, #Zoom a la imagen
)

# **4) Guardado de Modelo**

Después de un entrenamiento se realiza un guarado de modelo para poder utilizar luego o simplemente comparar

Guardado de modelo


In [ ]:
model.save('mi_modelo.pt')

Cargado de modelo

In [ ]:
mi_modelo = YOLO('mi_modelo.pt')

# **5) Realizar prediccion en produccion**

## **1) 🪐 Opción 1 Visualizar resultados**

In [ ]:
import os
from IPython.display import Image, display

Crear una carpeta para guardar resultados de prueba (Crear una carpeta para pasar imagenes y recibir resultados)

In [ ]:
save_dir = './testing/'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
img_path = '/content/defectos_de_soldaduras/valid/images/SampleV1_1_mp4-60_jpg.rf.f380bd888e3092f2864a3798d26b394a.jpg'

results = mi_modelo(img_path)

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_img_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_img_path))

## **2) 🪐 Opción 2 Filtrar la respuesta basandose en confianza**

In [ ]:
img_path = '/content/defectos_de_soldaduras/valid/images/SampleV1_2_mp4-5_jpg.rf.16bb7a236edddcaeb6d32772e9383599.jpg'

results = mi_modelo(img_path, conf=0.30) #Solo devuelve los de confianza 30%

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_img_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(filename=result_img_path))

## **3) 🪐 Opción 3 Filtrar por clase y confianza**

In [ ]:
img_path = '/content/defectos_de_soldaduras/valid/images/SampleV1_2_mp4-5_jpg.rf.16bb7a236edddcaeb6d32772e9383599.jpg'

results = mi_modelo(img_path, conf=0.30, classes=[0])

for result in results:
  result.plot(save=True, filename=os.path.join(save_dir, os.path.basename(result.path)))
  result_img_path = os.path.join(save_dir, os.path.basename(result.path))
  display(Image(result_img_path))

## **4) 🪐 Opción 4 Devolver informacion importante de la tedeccion**

In [ ]:
def detectar_objetos(image_path, model):
    """
    Realiza la detección de objetos en una imagen y devuelve:
      - Las cajas delimitadoras (bounding boxes) en formato [x1, y1, x2, y2].
      - Los nombres de las clases detectadas.
      - Los confidences de cada detección.
      - La imagen anotada con las detecciones.

    Parámetros:
      image_path (str): Ruta a la imagen.
      model: Modelo YOLO entrenado.

    Retorna:
      boxes (numpy.ndarray): Arreglo de bounding boxes.
      class_names (list): Lista con los nombres de las clases detectadas.
      confidences (numpy.ndarray): Arreglo con los scores/confidencias.
      annotated_img (numpy.ndarray): Imagen con las detecciones dibujadas.
    """
    # Realiza la detección (para una imagen, results es una lista y tomamos el primer resultado)
    results = model(image_path)
    result = results[0]

    # Extrae las cajas delimitadoras (bounding boxes) en formato xyxy
    boxes = result.boxes.xyxy.cpu().numpy()

    # Extrae los índices de las clases detectadas y las confidencias
    classes_idx = result.boxes.cls.cpu().numpy().astype(int)
    confidences = result.boxes.conf.cpu().numpy()

    # Si el modelo tiene el diccionario de nombres, se obtienen los nombres de las clases
    if hasattr(model, "names"):
        class_names = [model.names[i] for i in classes_idx]
    else:
        class_names = classes_idx  # Si no existe, se retorna los índices de clase

    # Se obtiene la imagen con los bounding boxes ya dibujados
    annotated_img = result.plot()

    return boxes, class_names, confidences, annotated_img

In [ ]:
 img_path = '/content/defectos_de_soldaduras/valid/images/SampleV2_1_mp4-48_jpg.rf.b34ab60a5e4bee8aec8425096027e52b.jpg'
 boxes, class_names, confidences, annotated_img = detectar_objetos(img_path, mi_modelo)

In [ ]:
annotated_img

In [ ]:
print('Otra informacion importante')
print(f'bbxes: {boxes}')
print(f'bbxes: {confidences}')
print(f'bbxes: {class_names}')